In [1]:
from ortools.linear_solver import pywraplp

In [2]:
import pickle
import numpy as np

# Loading the list from the pickle file
file_path = r"F:\dist_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    dist_list = pickle.load(file)

print("Dist list loaded successfully.")
print(np.shape(dist_list))

Dist list loaded successfully.
(400, 10450)


In [3]:
dist_list =  np.transpose(dist_list, (1, 0))

In [15]:
costs = [
    [90, 80, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 95],
    [45, 110, 95, 115],
    [60, 120, 50, 100],
    [50, 100, 90, 400],
    [50, 200, 170, 120],
    [50, 200, 190, 160]
]
num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in range(num_workers):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print(f"Worker {i} assigned to task {j}." + f" Cost: {costs[i][j]}")
else:
    print("No solution found.")


Total cost = 240.0

Worker 0 assigned to task 1. Cost: 80
Worker 1 assigned to task 3. Cost: 65
Worker 3 assigned to task 0. Cost: 45
Worker 4 assigned to task 2. Cost: 50


In [4]:
from tqdm import tqdm

costs = dist_list

num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in tqdm(range(num_workers)):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))

status = solver.Solve()

sol_indexes = []
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.1:
                print(f"Cluster {j} assigned to Class {i}." + f" Cost: {costs[i][j]}")
                sol_indexes.append(i)
else:
    print("No solution found.")

100%|██████████████████████████████████████████████████████████████████| 10450/10450 [02:42<00:00, 64.41it/s]


Total cost = 50126.7305385256

Cluster 89 assigned to Class 2. Cost: 122.04800441528762
Cluster 337 assigned to Class 22. Cost: 122.20200476340902
Cluster 24 assigned to Class 52. Cost: 134.37646000678754
Cluster 203 assigned to Class 89. Cost: 100.84299954478483
Cluster 188 assigned to Class 90. Cost: 93.20928674948505
Cluster 288 assigned to Class 91. Cost: 162.26941632770593
Cluster 1 assigned to Class 94. Cost: 108.22710969055342
Cluster 72 assigned to Class 95. Cost: 132.63127835401593
Cluster 269 assigned to Class 103. Cost: 98.46246838294753
Cluster 141 assigned to Class 140. Cost: 128.596831602121
Cluster 330 assigned to Class 280. Cost: 154.1071905700347
Cluster 196 assigned to Class 481. Cost: 116.59693325370243
Cluster 328 assigned to Class 484. Cost: 122.5652369678186
Cluster 88 assigned to Class 495. Cost: 151.43963679031103
Cluster 90 assigned to Class 496. Cost: 111.60341470195146
Cluster 198 assigned to Class 497. Cost: 95.56804287286857
Cluster 298 assigned to Class 50

Cluster 239 assigned to Class 4950. Cost: 109.05719809998621
Cluster 94 assigned to Class 4990. Cost: 199.55799197359718
Cluster 284 assigned to Class 4998. Cost: 174.88919100657068
Cluster 103 assigned to Class 5002. Cost: 141.38954425841433
Cluster 64 assigned to Class 5039. Cost: 112.39976735224958
Cluster 325 assigned to Class 5040. Cost: 107.44721099243382
Cluster 10 assigned to Class 5044. Cost: 140.38841297078855
Cluster 97 assigned to Class 5045. Cost: 106.08969532422427
Cluster 202 assigned to Class 5053. Cost: 94.85502590371266
Cluster 246 assigned to Class 5090. Cost: 134.43854492516323
Cluster 204 assigned to Class 5431. Cost: 143.29471005807656
Cluster 86 assigned to Class 5446. Cost: 114.6515146587214
Cluster 169 assigned to Class 5447. Cost: 109.20043966734283
Cluster 69 assigned to Class 5450. Cost: 127.86650838511322
Cluster 311 assigned to Class 5461. Cost: 173.9632699047028
Cluster 131 assigned to Class 5497. Cost: 96.54838245473005
Cluster 180 assigned to Class 5529

Cluster 382 assigned to Class 8446. Cost: 121.24949290372385
Cluster 197 assigned to Class 8447. Cost: 97.69892192756963
Cluster 129 assigned to Class 8450. Cost: 125.72714899825206
Cluster 251 assigned to Class 8497. Cost: 106.87873961504505
Cluster 317 assigned to Class 8529. Cost: 218.08611441668558
Cluster 191 assigned to Class 8546. Cost: 154.05093636645933
Cluster 388 assigned to Class 8552. Cost: 165.25265905567684
Cluster 217 assigned to Class 8554. Cost: 133.87834427826434
Cluster 377 assigned to Class 8566. Cost: 123.96119293589166
Cluster 282 assigned to Class 8568. Cost: 110.22054606620696
Cluster 358 assigned to Class 8902. Cost: 147.52093854843218
Cluster 356 assigned to Class 8904. Cost: 139.16178059962718
Cluster 384 assigned to Class 8925. Cost: 117.11848485473999
Cluster 161 assigned to Class 8950. Cost: 119.67054529194952
Cluster 118 assigned to Class 9011. Cost: 127.32072536957632
Cluster 32 assigned to Class 9012. Cost: 160.54135581656627
Cluster 174 assigned to Cl

In [7]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"F:\index_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(sol_indexes, file)

print("Combo list saved successfully.")


Combo list saved successfully.
